<a href="https://colab.research.google.com/github/thegallier/timeseries/blob/main/lead_lag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install esig
!pip install pywt
!pip install dtaidistance

ERROR: Could not find a version that satisfies the requirement pywt (from versions: none)
ERROR: No matching distribution found for pywt
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for dtaidistance: filename=dtaidistance-2.3.12-cp311-cp311-linux_x86_64.whl size=2628515 sha256=cb1bce401d036b53dd91fb98d3fb14b4926a3df5a8f531864f81e90404305a16
  Stored in directory: /root/.cache/pip/wheels/d9/3e/79/f71e79f61d63670c1f7cf9df3e474d9d9ccb35e56b547f8896
Successfully built dtaidistance


In [12]:
import numpy as np
import pandas as pd
from scipy.signal import correlate

###############################################################################
# 1) Real Data Creation
###############################################################################

def create_real_data(n=200, seed=42):
    """
    Creates an asynchronous orders DataFrame with columns:
      [datetime, symbol, price, quantity, side, orderType, data1, data2]

    'data1' and 'data2' are additional numeric fields for demonstration.
    We do NOT group by them. We'll group only by (symbol, side, orderType).
    """
    np.random.seed(seed)

    # Random exponential interarrival times
    start = pd.Timestamp("2023-01-01 09:30:00")
    random_gaps = np.random.exponential(scale=30, size=n).cumsum()  # ~30s average gap
    datetimes = start + pd.to_timedelta(random_gaps, unit='s')

    # Symbol
    symbols = np.random.choice(['AAPL','GOOG','TSLA'], size=n, p=[0.4,0.3,0.3])
    # Price as random walk
    price = 100 + np.cumsum(np.random.randn(n))
    # Quantity
    quantity = np.random.randint(1, 500, size=n)
    # Side
    side = np.random.choice(['buy','sell'], size=n)
    # orderType
    order_type = np.random.choice(['market','limit','stop'], size=n, p=[0.5,0.4,0.1])
    # data1, data2 (extra numeric columns)
    data1 = np.random.normal(loc=50, scale=10, size=n)
    data2 = np.random.normal(loc=100, scale=20, size=n)

    df = pd.DataFrame({
        'datetime': datetimes,
        'symbol': symbols,
        'price': price,
        'quantity': quantity,
        'side': side,
        'orderType': order_type,
        'data1': data1,
        'data2': data2
    }).sort_values('datetime')
    return df


###############################################################################
# 2) Synthetic Data Generation (Grouping only by symbol, side, orderType)
###############################################################################

def generate_synthetic_data(n=100,
                            reference_df=None,
                            use_features_from_reference=True,
                            seed=999):
    """
    Create a synthetic dataset with the same columns.
    We only group by ['symbol','side','orderType'] so the group key is a 3-tuple.
    We do NOT group by data1 or data2, but we DO store them from the chosen reference row.

    Avoids the mismatch in shape that causes:
        ValueError: must supply a tuple to get_group with multiple grouping keys
    """
    np.random.seed(seed)

    # We'll produce random ascending datetimes for synthetic
    start = pd.Timestamp("2023-01-02 09:30:00")
    random_gaps = np.random.exponential(scale=25, size=n).cumsum()
    synthetic_times = start + pd.to_timedelta(random_gaps, unit='s')

    if (reference_df is None) or (not use_features_from_reference):
        # Unconditional
        symbols = np.random.choice(['AAPL','GOOG','TSLA'], size=n)
        price = 100 + np.cumsum(np.random.randn(n))
        quantity = np.random.randint(1, 500, size=n)
        side = np.random.choice(['buy','sell'], size=n)
        order_type = np.random.choice(['market','limit','stop'], size=n, p=[0.5,0.4,0.1])
        data1 = np.random.normal(loc=50, scale=10, size=n)
        data2 = np.random.normal(loc=100, scale=20, size=n)

        df_synth = pd.DataFrame({
            'datetime': synthetic_times,
            'symbol': symbols,
            'price': price,
            'quantity': quantity,
            'side': side,
            'orderType': order_type,
            'data1': data1,
            'data2': data2
        }).sort_values('datetime')
        return df_synth

    else:
        # Group only by these 3 columns => each group_key is a 3-tuple
        groups = reference_df.groupby(['symbol','side','orderType'])
        group_keys = list(groups.groups.keys())
        # Must be object array, each element is a 3-tuple
        group_keys = np.array(group_keys, dtype=object)

        group_sizes = [len(groups.get_group(k)) for k in group_keys]
        total = sum(group_sizes)
        probs = [gs/total for gs in group_sizes]

        rows = []
        for i in range(n):
            dt = synthetic_times[i]
            choice_key = np.random.choice(group_keys, p=probs)
            # e.g. choice_key = ('AAPL','buy','limit'), a 3-tuple
            sub_df = groups.get_group(choice_key)
            # sample 1 row from that subgroup
            row_ref = sub_df.sample(n=1).iloc[0]
            # copy relevant fields from that row
            new_row = {
                'datetime': dt,
                'symbol': row_ref['symbol'],
                'price': row_ref['price'],
                'quantity': row_ref['quantity'],
                'side': row_ref['side'],
                'orderType': row_ref['orderType'],
                'data1': row_ref['data1'],  # we store the same data1, or random from subgroup
                'data2': row_ref['data2']
            }
            rows.append(new_row)

        df_synth = pd.DataFrame(rows).sort_values('datetime')
        return df_synth


###############################################################################
# 3) Example transforms and cross-correlation
###############################################################################

def transform_series(df, transform='returns', price_col='price'):
    """
    We'll produce 'value' column. Options: 'returns', 'log_returns', 'diff', 'none'
    Sort by datetime first (asynchronous).
    """
    df = df.sort_values('datetime').copy()
    if transform == 'returns':
        df['value'] = df[price_col].pct_change().fillna(0)
    elif transform == 'log_returns':
        df['value'] = np.log(df[price_col]).diff().fillna(0)
    elif transform == 'diff':
        df['value'] = df[price_col].diff().fillna(0)
    else:
        # no transform
        df['value'] = df[price_col]
    return df[['datetime','value']]


def cross_correlation_method(returns1, returns2, max_lag=20):
    """
    Row-based cross-correlation from your snippet
    (NOT time-based; best_lag is # of array elements).
    """
    cross_corr = correlate(returns1, returns2, mode='full')
    lags = np.arange(-len(returns1)+1, len(returns1))
    best_lag = lags[np.argmax(np.abs(cross_corr))]
    if abs(best_lag) > max_lag:
        best_lag = max_lag * np.sign(best_lag)
    return best_lag


###############################################################################
# DEMO
###############################################################################

def main():
    # 1) Create real data
    df_real = create_real_data(n=50, seed=42)
    print("Real data example:")
    print(df_real.head(5))

    # 2) Create synthetic data referencing real groups
    df_synth = generate_synthetic_data2(n=30, reference_df=df_real, use_features_from_reference=True, seed=123)
    print("\nSynthetic data example:")
    print(df_synth.head(5))

    # 3) Transform both => log_returns
    real_trans  = transform_series(df_real, transform='log_returns')
    synth_trans = transform_series(df_synth, transform='log_returns')

    # 4) Row-based cross-correlation on 'value' arrays
    arr_real  = real_trans['value'].values
    arr_synth = synth_trans['value'].values
    min_len = min(len(arr_real), len(arr_synth))
    best_lag_rows = cross_correlation_method(arr_real[:min_len], arr_synth[:min_len], max_lag=10)
    print(f"\nRow-based cross_correlation_method => best_lag_rows={best_lag_rows}")

if __name__=='__main__':
    main()


Real data example:
                       datetime symbol       price  quantity  side orderType  \
0 2023-01-01 09:30:14.078042699   TSLA  100.087047       240  sell    market   
1 2023-01-01 09:31:44.381685627   TSLA   99.788040       144  sell    market   
2 2023-01-01 09:32:23.884056433   TSLA   99.879800        97   buy    market   
3 2023-01-01 09:32:51.272333046   TSLA   97.892232       201   buy     limit   
4 2023-01-01 09:32:56.361079160   GOOG   97.672560       124  sell     limit   

       data1       data2  
0  54.497740  113.981848  
1  67.789243  111.997933  
2  43.330566   90.375037  
3  33.081817   58.124120  
4  56.567973   91.599051  

Synthetic data example:
                       datetime symbol      price  quantity  side orderType  \
0 2023-01-02 09:30:29.806803587   AAPL  96.994863       441   buy     limit   
1 2023-01-02 09:30:38.233490654   AAPL  98.989296       464   buy     limit   
2 2023-01-02 09:30:44.665592657   GOOG  98.181745         2   buy    market 

In [32]:
!pip install esig==0.6.9

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.5/69.5 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for esig: filename=esig-0.6.9-cp311-cp311-linux_x86_64.whl size=19298398 sha256=7eba863de8ae18265f7d7a706d144506bfbbec5cda72e6207ba3a974fbb6b386
  Stored in directory: /root/.cache/pip/wheels/27/39/83/e19f329e6cb503c72f6f1fecf7b11c936d3810c59739419a79
Successfully built esig
  Attempting uninstall: esig
    Found existing installation: esig 1.0.0
    Uninstalling esig-1.0.0:
      Successfully uninstalled esig-1.0.0


In [1]:
from esig import *

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Optional libraries:
try:
    from dtaidistance import dtw  # for DTW
    DTW_AVAILABLE = True
except ImportError:
    DTW_AVAILABLE = False

try:
    import PyWavelets as pywt  # for wavelets
    PYWT_AVAILABLE = True
except ImportError:
    PYWT_AVAILABLE = False

try:
    from esig import tosig  # for signatures
    ESIG_AVAILABLE = True
except ImportError:
    ESIG_AVAILABLE = False

###############################################################################
# 1) CORRECTED DATA GENERATION
###############################################################################

def create_real_data(n=200, seed=42):
    """
    Creates asynchronous orders with columns:
      [datetime, symbol, price, quantity, side, orderType, data1, data2].
    """
    np.random.seed(seed)

    # Random exponential interarrival times
    start = pd.Timestamp("2023-01-01 09:30:00")
    random_gaps = np.random.exponential(scale=30, size=n).cumsum()
    datetimes = start + pd.to_timedelta(random_gaps, unit='s')

    symbols = np.random.choice(['AAPL','GOOG','TSLA'], size=n, p=[0.4,0.3,0.3])
    price = 100 + np.cumsum(np.random.randn(n))
    quantity = np.random.randint(1, 500, size=n)
    side = np.random.choice(['buy','sell'], size=n)
    order_type = np.random.choice(['market','limit','stop'], size=n, p=[0.5,0.4,0.1])
    data1 = np.random.normal(loc=50, scale=10, size=n)
    data2 = np.random.normal(loc=100, scale=20, size=n)

    df = pd.DataFrame({
        'datetime': datetimes,
        'symbol': symbols,
        'price': price,
        'quantity': quantity,
        'side': side,
        'orderType': order_type,
        'data1': data1,
        'data2': data2
    })
    df.sort_values('datetime', inplace=True)
    return df


def generate_synthetic_data(n=100,
                            reference_df=None,
                            use_features_from_reference=True,
                            seed=999):
    """
    Synthetic dataset with same columns.
    Grouping only by (symbol, side, orderType) to avoid mismatch keys.
    """
    np.random.seed(seed)

    start = pd.Timestamp("2023-01-02 09:30:00")
    random_gaps = np.random.exponential(scale=25, size=n).cumsum()
    synth_times = start + pd.to_timedelta(random_gaps, unit='s')

    if (reference_df is None) or (not use_features_from_reference):
        # Unconditional
        symbols = np.random.choice(['AAPL','GOOG','TSLA'], size=n)
        price   = 100 + np.cumsum(np.random.randn(n))
        qty     = np.random.randint(1, 500, size=n)
        side    = np.random.choice(['buy','sell'], size=n)
        otype   = np.random.choice(['market','limit','stop'], size=n, p=[0.5,0.4,0.1])
        data1   = np.random.normal(loc=50,  scale=10,  size=n)
        data2   = np.random.normal(loc=100, scale=20,  size=n)

        df_synth = pd.DataFrame({
            'datetime': synth_times,
            'symbol': symbols,
            'price': price,
            'quantity': qty,
            'side': side,
            'orderType': otype,
            'data1': data1,
            'data2': data2
        })
        df_synth.sort_values('datetime', inplace=True)
        return df_synth

    else:
        # Group only by (symbol, side, orderType)
        groups = reference_df.groupby(['symbol','side','orderType'])
        group_keys = list(groups.groups.keys())
        group_keys = np.array(group_keys, dtype=object)  # must be 1D object array

        group_sizes = [len(groups.get_group(k)) for k in group_keys]
        total = sum(group_sizes)
        probs = [gs/total for gs in group_sizes]

        rows = []
        for i in range(n):
            dt = synth_times[i]
            choice_key = np.random.choice(group_keys, p=probs)
            sub_df = groups.get_group(choice_key)
            row_ref = sub_df.sample(n=1).iloc[0]

            new_row = {
                'datetime': dt,
                'symbol': row_ref['symbol'],
                'price': row_ref['price'],
                'quantity': row_ref['quantity'],
                'side': row_ref['side'],
                'orderType': row_ref['orderType'],
                'data1': row_ref['data1'],
                'data2': row_ref['data2']
            }
            rows.append(new_row)

        df_synth = pd.DataFrame(rows)
        df_synth.sort_values('datetime', inplace=True)
        return df_synth

###############################################################################
# 2) HELPER: TRANSFORM SERIES
###############################################################################

def transform_series(df, transform='log_returns', price_col='price'):
    """
    Create a 'value' column from 'price' using the chosen transform.
    Options: 'returns', 'log_returns', 'diff', 'none'
    """
    df = df.sort_values('datetime').copy()
    if transform == 'returns':
        df['value'] = df[price_col].pct_change().fillna(0)
    elif transform == 'log_returns':
        df['value'] = np.log(df[price_col]).diff().fillna(0)
    elif transform == 'diff':
        df['value'] = df[price_col].diff().fillna(0)
    else:
        # no transform
        df['value'] = df[price_col]
    return df[['datetime','value']]

###############################################################################
# 3) TIME-BASED RESAMPLING & SHIFT
###############################################################################

def resample_to_uniform_grid(df, base_time, fill_method='ffill'):
    """
    Convert 'datetime' -> ms from base_time, then create a Series
    with 1ms steps from min to max. Forward-fill missing.
    Return the (index array, series array).
    """
    df = df.copy()
    df['time_ms'] = (df['datetime'] - base_time).dt.total_seconds()*1000
    t_min = int(df['time_ms'].min())
    t_max = int(df['time_ms'].max())
    idx = np.arange(t_min, t_max+1, 1, dtype=int)

    s = pd.Series(index=idx, dtype=float)
    for (tm,val) in zip(df['time_ms'], df['value']):
        s.loc[int(tm)] = val

    if fill_method:
        s = s.fillna(method='ffill').fillna(0)
    return idx, s.values  # time index array, data array

###############################################################################
# 4) CROSS-CORRELATION (TIME-BASED)
###############################################################################

def find_lead_lag_crosscorr_time(df1, df2, max_lag_ms=2000, step_ms=100):
    """
    For cross-correlation in time. Shift df2 from -max_lag..+max_lag by step_ms,
    measure Pearson correlation, pick best. Return (best_lag_ms, best_corr).
    """
    # 1) find a common base_time
    base_t = min(df1['datetime'].min(), df2['datetime'].min())
    # 2) uniform grid
    idx1, arr1 = resample_to_uniform_grid(df1, base_t)
    idx2, arr2 = resample_to_uniform_grid(df2, base_t)

    best_lag, best_corr = 0, -999

    for lag in range(-max_lag_ms, max_lag_ms + 1, step_ms):
        # Ensure a1 and a2 have the same length after shifting
        len_a1 = len(arr1) - abs(lag)  # Length of a1 after potential truncation
        len_a2 = len(arr2) - abs(lag)  # Length of a2 after potential truncation
        min_len = min(len_a1, len_a2)  # Take the minimum length

        if lag > 0:
            a1 = arr1[lag : lag + min_len]
            a2 = arr2[:min_len]
        elif lag < 0:
            a1 = arr1[:min_len]
            a2 = arr2[abs(lag) : abs(lag) + min_len]
        else:
            a1 = arr1[:min_len]
            a2 = arr2[:min_len]

        if len(a1) < 2 or len(a2) < 2:
            continue

        c = np.corrcoef(a1, a2)[0, 1]
        if abs(c) > abs(best_corr):
            best_corr = c
            best_lag = lag
    return best_lag, best_corr

###############################################################################
# 5) DTW (TIME-BASED)
###############################################################################

def find_lead_lag_dtw_time(df1, df2, max_lag_ms=2000, step_ms=100):
    """
    For each shift in [-max_lag_ms..+max_lag_ms], measure DTW distance
    after uniform resampling. Return (best_lag_ms, best_dist).
    Minimizes DTW distance.
    """
    if not DTW_AVAILABLE:
        print("DTW library not installed. Returning None.")
        return None, None

    base_t = min(df1['datetime'].min(), df2['datetime'].min())
    idx1, arr1 = resample_to_uniform_grid(df1, base_t)
    idx2, arr2 = resample_to_uniform_grid(df2, base_t)

    best_lag, best_dist = 0, 999999999

    for lag in range(-max_lag_ms, max_lag_ms+1, step_ms):
        if lag > 0:
            a1 = arr1[lag:]
            a2 = arr2[:len(a1)]
        elif lag < 0:
            l_ = abs(lag)
            a1 = arr1[:len(arr1)-l_]
            a2 = arr2[l_:l_+len(a1)]
        else:
            a1 = arr1
            a2 = arr2

        if len(a1) < 2 or len(a2) < 2:
            continue

        dist = dtw.distance(a1, a2)
        if dist < best_dist:
            best_dist = dist
            best_lag = lag
    return best_lag, best_dist

###############################################################################
# 6) HAYASHI–YOSHIDA (TIME-BASED SHIFT)
###############################################################################

def hayashi_yoshida_covar(arrA, arrB, idxA, idxB):
    """
    Naive HY covariance on two signals (arrA, arrB) with time indexes in ms (idxA, idxB).
    We'll interpret each increment in arrA, arrB.
    This is simplistic; real HY is typically on tick data.
    """
    # increments for A
    A_incr = []
    for i in range(1, len(arrA)):
        dX = arrA[i] - arrA[i-1]
        t0A, t1A = idxA[i-1], idxA[i]
        A_incr.append((t0A, t1A, dX))

    # increments for B
    B_incr = []
    for j in range(1, len(arrB)):
        dY = arrB[j] - arrB[j-1]
        t0B, t1B = idxB[j-1], idxB[j]
        B_incr.append((t0B, t1B, dY))

    hy_sum = 0.0
    for (A0, A1, dX) in A_incr:
        for (B0, B1, dY) in B_incr:
            if A1 > B0 and B1 > A0:  # intervals overlap
                hy_sum += dX * dY
    return hy_sum

def find_lead_lag_hy_time(df1, df2, max_lag_ms=2000, step_ms=100):
    """
    Shift df2 in [-max_lag_ms..+max_lag_ms], compute HY correlation, find the shift with highest corr.
    Return (best_lag_ms, best_corr).
    """
    base_t = min(df1['datetime'].min(), df2['datetime'].min())
    idx1, arr1 = resample_to_uniform_grid(df1, base_t)
    idx2, arr2 = resample_to_uniform_grid(df2, base_t)

    best_lag, best_corr = 0, -999

    # We'll define a helper to compute HY correlation
    def hy_correlation(aA, iA, aB, iB):
        covAB = hayashi_yoshida_covar(aA, aB, iA, iB)
        varA  = hayashi_yoshida_covar(aA, aA, iA, iA)
        varB  = hayashi_yoshida_covar(aB, aB, iB, iB)
        if varA <= 0 or varB <= 0:
            return 0
        return covAB / np.sqrt(varA*varB)

    for lag in range(-max_lag_ms, max_lag_ms+1, step_ms):
        if lag > 0:
            a1 = arr1[lag:]
            i1 = idx1[lag:]
            a2 = arr2[:len(a1)]
            i2 = idx2[:len(a1)]
        elif lag < 0:
            l_ = abs(lag)
            a1 = arr1[:len(arr1)-l_]
            i1 = idx1[:len(arr1)-l_]
            a2 = arr2[l_:l_+len(a1)]
            i2 = idx2[l_:l_+len(a1)]
        else:
            a1 = arr1
            i1 = idx1
            a2 = arr2
            i2 = idx2

        if len(a1) < 2 or len(a2) < 2:
            continue

        corr = hy_correlation(a1, i1, a2, i2)
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag

    return best_lag, best_corr

###############################################################################
# 7) WAVELET (TIME-BASED SHIFT)
###############################################################################

def find_lead_lag_wavelet_time(df1, df2, max_lag_ms=2000, step_ms=100, wavelet='morl', scales=range(1,6)):
    """
    Shift df2, compute wavelet transform of both. Then measure correlation
    between summed wavelet coeffs. Return shift that yields highest correlation in absolute value.
    """
    if not PYWT_AVAILABLE:
        print("PyWavelets not installed. Returning None.")
        return None, None

    base_t = min(df1['datetime'].min(), df2['datetime'].min())
    idx1, arr1 = resample_to_uniform_grid(df1, base_t)
    idx2, arr2 = resample_to_uniform_grid(df2, base_t)

    def wavelet_feats(sig):
        # compute CWT, sum abs coefficients
        coeffs, freqs = pywt.cwt(sig, scales, wavelet)
        # shape = (len(scales), len(sig))
        return np.sum(np.abs(coeffs), axis=1)  # 1D array of length=len(scales)

    best_lag, best_corr = 0, -999

    for lag in range(-max_lag_ms, max_lag_ms+1, step_ms):
        if lag > 0:
            a1 = arr1[lag:]
            a2 = arr2[:len(a1)]
        elif lag < 0:
            l_ = abs(lag)
            a1 = arr1[:len(arr1)-l_]
            a2 = arr2[l_:l_+len(a1)]
        else:
            a1 = arr1
            a2 = arr2

        if len(a1) < 2 or len(a2) < 2:
            continue

        w1 = wavelet_feats(a1)
        w2 = wavelet_feats(a2)

        # correlation of wavelet feature vectors ( length=len(scales) )
        c = np.corrcoef(w1, w2)[0,1]
        if abs(c) > abs(best_corr):
            best_corr = c
            best_lag = lag
    return best_lag, best_corr

###############################################################################
# 8) SIGNATURE (TIME-BASED SHIFT)
###############################################################################

def find_lead_lag_signature_time(df1, df2, max_lag_ms=2000, step_ms=100, level=2):
    """
    Shift df2, compute signature for each series, measure correlation of signature vectors.
    Return best shift.
    Only works if 'esig' is installed.
    """
    if not ESIG_AVAILABLE:
        print("esig not installed. Returning None.")
        return None, None

    base_t = min(df1['datetime'].min(), df2['datetime'].min())
    idx1, arr1 = resample_to_uniform_grid(df1, base_t)
    idx2, arr2 = resample_to_uniform_grid(df2, base_t)

    # Helper to compute signature up to 'level'
    def compute_sig(signal):
        # shape => (len, 1) for esig
        path = signal.reshape(-1,1)
        sig = tosig.stream2sig(path, level)
        return sig

    best_lag, best_corr = 0, -999

    for lag in range(-max_lag_ms, max_lag_ms+1, step_ms):
        if lag > 0:
            a1 = arr1[lag:]
            a2 = arr2[:len(a1)]
        elif lag < 0:
            l_ = abs(lag)
            a1 = arr1[:len(arr1)-l_]
            a2 = arr2[l_:l_+len(a1)]
        else:
            a1 = arr1
            a2 = arr2

        if len(a1) < 2 or len(a2) < 2:
            continue

        s1 = compute_sig(a1)
        s2 = compute_sig(a2)
        # measure correlation of signature vectors (use Pearson or cos-sim)
        if np.std(s1) < 1e-15 or np.std(s2) < 1e-15:
            continue
        c = np.corrcoef(s1, s2)[0,1]
        if abs(c) > abs(best_corr):
            best_corr = c
            best_lag = lag

    return best_lag, best_corr

###############################################################################
# 9) DEMO
###############################################################################

def main():
    # A) Create real data
    df_real = create_real_data(n=100, seed=42)
    # B) Create synthetic referencing real subgroups
    df_synth = generate_synthetic_data2(n=80, reference_df=df_real, use_features_from_reference=True, seed=123)

    # Choose transformations (e.g. log_returns)
    df_real_trans  = transform_series(df_real, transform='log_returns')
    df_synth_trans = transform_series(df_synth, transform='log_returns')

    # CROSS-CORRELATION
    best_lag_cc, best_corr_cc = find_lead_lag_crosscorr_time(df_real_trans, df_synth_trans,
                                                             max_lag_ms=2000, step_ms=100)
    print(f"[CrossCorr] best_lag={best_lag_cc} ms, corr={best_corr_cc:.4f}")

    # DTW
    if DTW_AVAILABLE:
        best_lag_dtw, best_dist_dtw = find_lead_lag_dtw_time(df_real_trans, df_synth_trans,
                                                             max_lag_ms=2000, step_ms=100)
        print(f"[DTW] best_lag={best_lag_dtw} ms, distance={best_dist_dtw:.4f}")

    # Hayashi-Yoshida
    best_lag_hy, best_corr_hy = find_lead_lag_hy_time(df_real_trans, df_synth_trans,
                                                      max_lag_ms=2000, step_ms=100)
    print(f"[HY] best_lag={best_lag_hy} ms, corr={best_corr_hy:.4f}")

    # Wavelet
    if PYWT_AVAILABLE:
        best_lag_wave, best_corr_wave = find_lead_lag_wavelet_time(df_real_trans, df_synth_trans,
                                                                   max_lag_ms=2000, step_ms=100,
                                                                   wavelet='morl', scales=range(1,6))
        print(f"[Wavelet] best_lag={best_lag_wave} ms, corr={best_corr_wave:.4f}")

    # Signature
    if ESIG_AVAILABLE:
        best_lag_sig, best_corr_sig = find_lead_lag_signature_time(df_real_trans, df_synth_trans,
                                                                   max_lag_ms=2000, step_ms=100, level=2)
        print(f"[Signature] best_lag={best_lag_sig} ms, corr={best_corr_sig:.4f}")

if __name__ == "__main__":
    main()

<ipython-input-4-e01adb05829d>:174: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  s = s.fillna(method='ffill').fillna(0)
<ipython-input-4-e01adb05829d>:174: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  s = s.fillna(method='ffill').fillna(0)


[CrossCorr] best_lag=0 ms, corr=-999.0000


<ipython-input-4-e01adb05829d>:174: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  s = s.fillna(method='ffill').fillna(0)
<ipython-input-4-e01adb05829d>:174: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  s = s.fillna(method='ffill').fillna(0)


Comparison of Lead–Lag Models

Below is a high-level comparison of multiple methods for estimating lead–lag relationships (in milliseconds) between two time series. Each method has different strengths in terms of performance, noise-robustness, and ability to handle asynchronous data. We also provide references to two relevant papers and links to packages that implement or support these methods.

1. Cross-Correlation

Overview
	•	A classical measure that shifts one series in time (positive or negative lag) and computes the Pearson correlation at each shift.
	•	Identifies the shift (lag) that maximizes correlation (or minimizes negative correlation).

Performance & Noise
	•	Relatively fast (simple correlation operations).
	•	Linear measure, so outliers and heavy noise can reduce effectiveness; pre-filtering often helps.

Asynchronous Data
	•	Requires resampling both time series onto a common time grid. Loss of fine-grained detail if data is highly asynchronous.

Relevant Package
	•	Part of standard libraries: NumPy/numpy.correlate or SciPy/scipy.signal.correlate.

2. Dynamic Time Warping (DTW)

Overview
	•	Aligns two time series by warping the time axis to minimize overall distance.
	•	Unlike cross-correlation, DTW can handle local misalignments.

Performance & Noise
	•	Slower than cross-correlation, especially for large datasets (DTW is ￼ unless using advanced optimizations).
	•	Can handle moderate levels of noise, but strong outliers might distort the distance measure; options like a Sakoe–Chiba band can improve robustness.

Asynchronous Data
	•	Naturally handles asynchronous or non-uniformly sampled signals by warping one series to the other.

Relevant Package
	•	dtaidistance on PyPI implements DTW in Python (including acceleration and visualization).

3. Hayashi–Yoshida (HY)

Overview
	•	A finance-oriented approach that estimates integrated covariance or correlation for asynchronous time series.
	•	Summation of overlapping increments captures how two assets co-move at high frequency without strict alignment.

Performance & Noise
	•	Moderate computational effort in naive form (￼ for increment overlaps).
	•	Designed for stochastic processes in finance, so it’s somewhat robust to market microstructure noise, but large outliers still impact the increment sums.

Asynchronous Data
	•	Excellent: The HY method was specifically developed to handle irregular, tick-by-tick (asynchronous) data without resampling.

Relevant Package
	•	Typically implemented as custom code. See hyCov package (R) for a partial reference (though it’s not identical). Python versions are often user-coded.

4. Wavelet Approach

Overview
	•	Transforms each time series into wavelet coefficients (multi-scale features).
	•	Measures correlation (or distance) of these coefficients across time shifts.

Performance & Noise
	•	Wavelets can handle nonstationarity and multi-scale noise well.
	•	Computational cost depends on transform length and the wavelet scales used. Typically faster than DTW for large data.

Asynchronous Data
	•	Still requires time alignment or resampling to a common grid.
	•	Wavelet transforms can handle missing data (with interpolation or partial reconstruction), but not inherently an asynchronous method.

Relevant Package
	•	PyWavelets is the standard Python library for wavelet transforms.

5. Signature (Rough Paths)

Overview
	•	Uses rough path theory to encode time series as signatures (iterated integrals).
	•	The lead–lag can be inferred by shifting one series and measuring some distance or correlation between their signature vectors.

Performance & Noise
	•	Signatures can capture complex path-dependent features, somewhat robust to moderate noise.
	•	Computation grows exponentially in dimension/level, but for 1D or small dimension at low signature levels, it’s manageable.

Asynchronous Data
	•	In principle, signatures can handle asynchronous time series by carefully constructing the piecewise or step path.
	•	Typically requires user code or a library (like esig) to handle lead–lag transformations.

Relevant Package
	•	esig (Github) is a Python library for signatures.
	•	For an introduction, see this short doc.

Papers & References
	1.	Sakoe, H., & Chiba, S. (1978). Dynamic programming algorithm optimization for spoken word recognition. IEEE Transactions on Acoustics, Speech, and Signal Processing, 26(1), 43–49.
	•	Classic foundation for DTW-based time series alignment.
	2.	Hayashi, T., & Yoshida, N. (2005). On covariation estimation of non-synchronously observed diffusion processes. Bernoulli, 11(2), 359–379.
	•	Seminal paper for the Hayashi–Yoshida estimator of covariance with asynchronous data in finance.

(For wavelets, see also Daubechies, I. (1992). Ten Lectures on Wavelets. SIAM, which introduced the standard wavelet frameworks. For signatures, see Lyons, T. (1998). Differential equations driven by rough paths, in Ecole d’été de Probabilités de Saint-Flour. )

Summary
	•	Cross-Correlation is fast and straightforward but can struggle if data is asynchronous unless carefully resampled.
	•	DTW robustly aligns local time differences, good for moderate noise, but can be slow (￼) for large ￼.
	•	Hayashi–Yoshida is the go-to for finance tick data, elegantly handles asynchrony but can be ￼ in naive form.
	•	Wavelet approaches decompose data across multiple scales, handle nonstationarity, but still need time alignment.
	•	Signature expansions (rough paths) capture nonlinear path features, can handle asynchrony by a lead–lag construction, but are more specialized and can grow in complexity.

Each method has trade-offs in performance, noise handling, and asynchrony support. The choice often depends on the domain, data frequency, and the desired computational budget.

In [3]:
def generate_synthetic_data2(n=100,
                            reference_df=None,
                            use_features_from_reference=True,
                            seed=999):
    """
    Create a synthetic dataset with the same columns.
    We only group by ['symbol','side','orderType'] so the group key is a 3-tuple.
    We do NOT group by data1 or data2, but we DO store them from the chosen reference row.

    Avoids the mismatch in shape that causes:
        ValueError: must supply a tuple to get_group with multiple grouping keys
    """
    np.random.seed(seed)

    # We'll produce random ascending datetimes for synthetic
    start = pd.Timestamp("2023-01-02 09:30:00")
    random_gaps = np.random.exponential(scale=25, size=n).cumsum()
    synthetic_times = start + pd.to_timedelta(random_gaps, unit='s')

    if (reference_df is None) or (not use_features_from_reference):
        # Unconditional
        symbols = np.random.choice(['AAPL','GOOG','TSLA'], size=n)
        price = 100 + np.cumsum(np.random.randn(n))
        quantity = np.random.randint(1, 500, size=n)
        side = np.random.choice(['buy','sell'], size=n)
        order_type = np.random.choice(['market','limit','stop'], size=n, p=[0.5,0.4,0.1])
        data1 = np.random.normal(loc=50, scale=10, size=n)
        data2 = np.random.normal(loc=100, scale=20, size=n)

        df_synth = pd.DataFrame({
            'datetime': synthetic_times,
            'symbol': symbols,
            'price': price,
            'quantity': quantity,
            'side': side,
            'orderType': order_type,
            'data1': data1,
            'data2': data2
        }).sort_values('datetime')
        return df_synth

    else:
        # Group only by these 3 columns => each group_key is a 3-tuple
        groups = reference_df.groupby(['symbol','side','orderType'])
        group_keys = list(groups.groups.keys())  # Keep group_keys as a list of tuples

        group_sizes = [len(groups.get_group(k)) for k in group_keys]
        total = sum(group_sizes)
        probs = [gs/total for gs in group_sizes]

        rows = []
        for i in range(n):
            dt = synthetic_times[i]
            # Sample from the range of group indices
            choice_index = np.random.choice(range(len(group_keys)), p=probs)
            # Get the actual group key using the sampled index
            choice_key = group_keys[choice_index]

            sub_df = groups.get_group(choice_key)
            # sample 1 row from that subgroup
            row_ref = sub_df.sample(n=1).iloc[0]
            # copy relevant fields from that row
            new_row = {
                'datetime': dt,
                'symbol': row_ref['symbol'],
                'price': row_ref['price'],
                'quantity': row_ref['quantity'],
                'side': row_ref['side'],
                'orderType': row_ref['orderType'],
                'data1': row_ref['data1'],  # we store the same data1, or random from subgroup
                'data2': row_ref['data2']
            }
            rows.append(new_row)

        df_synth = pd.DataFrame(rows).sort_values('datetime')
        return df_synth